# Daily Credit Card Sales Prediction

Data Source: NH Card (2020~2024.2Q)

이 과제는 KDX 한국데이터거래소에서 제공하는 데이터를 활용하여 서울 지역 일별 카드 소비현황을 예측한다.

실제 업무 환경에서는 DW 및 Hadoop에서 SQL을 사용해 데이터를 추출하는 것부터 시작한다. 그러나 해당 데이터를 사용할 수 없어 KDX 한국데이터거래소를 통해 제공되는 NH농협카드의 일별 소비 데이터를 사용한다.

## 환경 설정

In [1]:
# %% 0. Environment Settings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.style.use('ggplot')

plt.rc('font', family='Malgun Gothic')
# plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

## 데이터 로드

데이터는 2020년 1월부터 2024년 6월까지의 일별 소비 데이터를 포함한다. 각 월별 CSV 파일로 제공되며 데이터를 하나의 DataFrame으로 통합한다.

* 훈련 데이터: 2020.01 ~ 2023.12
* 검증 데이터: 2024.01 ~ 2024.06

In [8]:
# Data Source: KDX Data - [NH농협카드] 일자별 소비현황_서울
df = pd.DataFrame()

bas_ym = pd.date_range(start='20200101', end='20240630', freq='MS').strftime('%Y%m').tolist()

for i, var in enumerate(bas_ym):
    data_path = f'data/[NH농협카드] 일자별 소비현황_서울_{var}.csv'
    
    encodings = ['utf-8-sig', 'euc-kr', 'cp949']
    for encoding in encodings:
        try:
            tmp_df = pd.read_csv(data_path, encoding=encoding)
            break
        except UnicodeDecodeError:
            continue
    else:
        raise ValueError(f"Failed to read {data_path} with available encodings.")
    
    df = pd.concat([df, tmp_df], axis=0)
    
print(df.shape)
print(df.head())


(1643, 8)
   시도      승인일자  이용건수_전체  이용금액_전체  이용건수_개인  이용금액_개인  이용건수_법인  이용금액_법인
0  서울  20200101   1827.0  48116.0   1770.0  45654.0     57.0   2462.0
1  서울  20200102   2071.0  78042.0   1932.0  66654.0    139.0  11388.0
2  서울  20200103   2140.0  77224.0   1995.0  64302.0    144.0  12922.0
3  서울  20200104   1969.0  55070.0   1891.0  51702.0     78.0   3369.0
4  서울  20200105   1711.0  46359.0   1649.0  43796.0     62.0   2563.0


동일 출처의 자료여도 인코딩 포맷이 달랐다. (중간에 담당자가 변경된 것일까?) 따라서 'utf-8-sig', 'euc-kr', 'cp949' 다양한 인코딩을 시도했다.

In [9]:
# Type Conversion (int64 -> datetime64)
df['date'] = pd.to_datetime(df['승인일자'], format='%Y%m%d')

# Decimal Point Handling
df['이용금액_전체'] = df['이용금액_전체'] / 100
df['이용금액_개인'] = df['이용금액_개인'] / 100
df['이용금액_법인'] = df['이용금액_법인'] / 100

# Derived Variables
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek

df['avg_sales'] = df['이용금액_전체'] * 1000000 / df['이용건수_전체'] / 1000
df['avg_sales_psn'] = df['이용금액_개인'] * 1000000 / df['이용건수_개인'] / 1000
df['avg_sales_cor'] = df['이용금액_법인'] * 1000000 / df['이용건수_법인'] / 1000

# Nominal to Ordinal Variable
df['dayname'] = pd.Categorical(df['date'].dt.day_name(), 
                               categories=['Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday'],
                               ordered=True)

# Add weekend variable
df['weekend'] = df['dayname'].isin(['Saturday', 'Sunday'])

# Nullity Check
print(df.isna().sum())

# Train-Test Split
df_train = df[df['year'] != 2024]
df_test = df[df['year'] == 2024]

print(df_train.shape, df_test.shape)

# Reset Index
df.reset_index(drop=True, inplace=True)

시도               0
승인일자             0
이용건수_전체          0
이용금액_전체          0
이용건수_개인          0
이용금액_개인          0
이용건수_법인          0
이용금액_법인          0
date             0
year             0
month            0
day              0
dayofweek        0
avg_sales        0
avg_sales_psn    0
avg_sales_cor    0
dayname          0
weekend          0
dtype: int64
(1461, 18) (182, 18)


* 주요 독립변수: 년(year), 월(month), 일(day), 요일(dayofweek)

신용카드 결제는 

In [ ]:
from sklearn.model_selection import train_test_split

# 4-1. Train-Validation Split
ind_vars = ['승인일자', 'year', 'month', 'day', 'dayofweek']
dep_vars = ['이용건수_개인', '이용건수_법인', '이용금액_개인', '이용금액_법인']

X = df[df['year'] != 2024][ind_vars]
y = df[df['year'] != 2024][dep_vars]

X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      test_size=0.2, 
                                                      random_state=42
                                                      )
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)


(1168, 5) (1168, 4)
(293, 5) (293, 4)


In [ ]:
# %% 4. Regression Model Specification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 4-2. Machine Learning (RandomForest, XGBoost, LightGBM)
dep_var = '이용금액_개인'

model_rf = RandomForestRegressor(n_estimators=1000,
                                max_depth=20,
                                random_state=42)
model_rf.fit(X_train, y_train[dep_var])

model_xgb = XGBRegressor(random_state=42)
model_xgb.fit(X_train, y_train[dep_var]) 

model_lgbm = LGBMRegressor(random_state=42)
model_lgbm.fit(X_train, y_train[dep_var])


# 4-3. Model Comparison
from sklearn.metrics import mean_squared_error

y_sample = y_valid[dep_var]

y_pred_rf = model_rf.predict(X_valid)
mse_rf = mean_squared_error(y_pred_rf, y_sample)
print(f'MSE (RandomForest): {mse_rf:.2f}')

y_pred_xgb = model_xgb.predict(X_valid)
mse_xgb = mean_squared_error(y_pred_xgb, y_sample)
print(f'MSE (XGBoost): {mse_xgb:.2f}')

y_pred_lgbm = model_lgbm.predict(X_valid)
mse_lgbm = mean_squared_error(y_pred_lgbm, y_sample)
print(f'MSE (LightGBM): {mse_lgbm:.2f}')

c:\Users\Red\miniconda3\envs\dl\lib\site-packages\joblib\externals\loky\backend\context.py:152: UserWarning: Could not find the number of physical cores for the following reason:
__init__() got an unexpected keyword argument 'capture_output'
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  "following reason:\n" + str(exception) + "\n"
  File "c:\Users\Red\miniconda3\envs\dl\lib\site-packages\joblib\externals\loky\backend\context.py", line 229, in _count_physical_cores
    capture_output=True)
  File "c:\Users\Red\miniconda3\envs\dl\lib\subprocess.py", line 423, in run
    with Popen(*popenargs, **kwargs) as process:


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000460 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 312
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 5
[LightGBM] [Info] Start training from score 738.366739
MSE (RandomForest): 13813.88
MSE (XGBoost): 11969.03
MSE (LightGBM): 9832.48
